In [2]:
pip install imagecodecs

     |████████████████████████████████| 17.9 MB 3.1 MB/s eta 0:00:01    |███                             | 1.7 MB 3.1 MB/s eta 0:00:06
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import cv2
import skimage.io
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import gc
import torch
import torch.nn as nn
import imagecodecs

In [12]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [23]:
def tile(name):
    sz = 128
    N = 16
    img = skimage.io.MultiImage(os.path.join('../input/prostate-cancer-grade-assessment/train_images/',name+'.tiff'))[-1]
    result = []
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    for i in range(len(img)):
        result.append({'img':preprocess(cv2.imencode('.png',cv2.cvtColor(img[i], cv2.COLOR_RGB2BGR))[1]), 'idx':i})
    return result

In [20]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torch.load("../input/modelp/PandaModel.pth")
        self.model.eval()
        
    def forward(self, name):
        result = []
        for img in tile(name):
            img = img.get('img')
            tensor = img.squeeze(-1).to("cuda")

            result.append(torch.argmax(self.model(tensor)))
        result = np.array(result)
        mc = np.bincount(result).argmax()
        second = np.bincount(result[~np.isin(result,mc)]).argmax()
        greatest = np.amax(result)
        if(mc == 0):
            return 0
        else:
            if (second == 0):
                return mc + mc
        if(mc == greatest):
            return greatest + second
        else:
            return greatest + mc
            
        
            

      


In [24]:
model = Network()
model.eval()
correct = 0
df = pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/train.csv')
for row,z in zip(df.iterrows(),tqdm(range(len(df)))):
    name = row[1].get(0)
    score = row[1].get(2)
    if(score == 1):
        score = 6
    else: 
        if score == 2:
            score = 7
        else: 
            if score == 3:
                score = 7
            else: 
                if score == 4:
                    score = 8
                else: 
                    if score == 5:
                        score = 10
    prediction = model(name)
    if(prediction == 9):
        prediction = 10
    if(prediction == score):
        correct = correct + 1
print("Accuracy: ",correct/len(df)*100,"%")


torch.save("/kaggle/output/finalModel.pth",model)





[[137]
 [ 80]
 [ 78]
 ...
 [ 66]
 [ 96]
 [130]]



TypeError: img should be PIL Image. Got <class 'numpy.ndarray'>